In [ ]:
import os
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib

In [ ]:
train_csv_path = '../data/train.csv'
test_csv_path = '../data/test.csv'
train_audio_dir = '../data/audios/train'
test_audio_dir = '../data/audios/test'

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

print("Train Data Sample:")
print(train_df.head())
print("Test Data Sample:")
print(test_df.head())

In [ ]:
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y=audio)
    rmse = librosa.feature.rms(y=audio)
    
    features = np.hstack([
        np.mean(mfcc, axis=1),
        np.mean(chroma, axis=1),
        np.mean(contrast, axis=1),
        np.mean(zcr),
        np.mean(rmse)
    ])
    return features

In [ ]:
X_train = []
y_train = []

for idx, row in train_df.iterrows():
    file_path = os.path.join(train_audio_dir, row['filename'])
    features = extract_features(file_path)
    X_train.append(features)
    y_train.append(row['label'])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

model = RandomForestRegressor(n_estimators=150, random_state=42)
model.fit(X_train_scaled, y_train)

train_preds = model.predict(X_train_scaled)
train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
print(f"Training RMSE: {train_rmse:.4f}")

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_rmse = np.sqrt(-cross_val_score(model, X_train_scaled, y_train, cv=kf, scoring='neg_mean_squared_error'))
print(f"Cross-validated RMSE: {cv_rmse.mean():.4f}")

In [ ]:
X_test = []
test_filenames = []

for idx, row in test_df.iterrows():
    file_path = os.path.join(test_audio_dir, row['filename'])
    features = extract_features(file_path)
    X_test.append(features)
    test_filenames.append(row['filename'])

X_test = np.array(X_test)
X_test_scaled = scaler.transform(X_test)

In [ ]:
test_preds = model.predict(X_test_scaled)

submission = pd.DataFrame({
    'filename': test_filenames,
    'label': test_preds
})
submission_path = '../submission.csv'
submission.to_csv(submission_path, index=False)
print(f"Submission file saved to: {submission_path}")

In [ ]:
model_path = '../grammar_scoring_model.pkl'
joblib.dump(model, model_path)
print(f"Trained model saved to: {model_path}")